### 每個政治的url,2016/11/26~now()

In [ ]:

import requests
from bs4 import BeautifulSoup
import re 
import datetime
import concurrent.futures
from datetime import datetime, timedelta, date
import json
udn_url=[]

_format = "%Y/%m/%d"
d = datetime.strptime("2016/11/25", _format) ##2016/11/25 00:00:00
delta = timedelta(days = 1)
today = datetime.strptime(date.today().strftime(_format), _format)
while d <= today:
    oldtime = d.strftime(_format)
    resp = requests.get("https://udn.com/news/archive/2/6638/%s/1"%oldtime) ##把時分秒去掉
    soup = BeautifulSoup(resp.text,"html5lib")
    selects = soup.select("div.pagelink > span.total")
    s = selects[0].text
    r = re.findall("共 (.) 頁",s)[0]
    r = int(r)
    print(r)
    for pg in range(1,r+1):
        URL = "https://udn.com/news/archive/2/6638/{}/{}".format(oldtime,pg)
        udn_url.append(URL)
    d += delta
print(udn_url)
with open("udn_url.txt","w") as f:
    f.write(json.dumps(udn_url))

### 爬每一個新聞的url

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor 
with open("udn_url.txt","r") as f:
    r = f.read()
    
udn_url = r.replace('"','').replace("[","").replace("]","").split(",")
def udn_news(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,"html5lib")
    selects = soup.select("tr > td > a")
    Domain = "https://udn.com"
    list_url=[]
    for s in selects :
        if s.get("href").startswith("/news/story/6656/"):
            u = s.get("href")
            list_url.append(Domain+u)
    return list_url
            
list_newss=[]        
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    for data in executor.map(udn_news, udn_url):
        print(data)
        list_newss.extend(data)
        
with open("udn_everynews_url.txt","w") as f:
    f.write(json.dumps(list_newss))

### 爬每個新聞標題、時間、內容

In [92]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
with open("udn_everynews_url.txt","r") as f:
    r = f.read()
udn_everynews_url = r.replace('"','').replace("[","").replace("]","").split(",")

def udn_everynews(url): 
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,"html5lib")
    try:
        
        dic = {}
        selects_title = soup.select("div#story_body_content > h1")
        dic["title"] = selects_title[0].text

        selects_time = soup.select("div.story_bady_info_author > span")
        dic["time"] = selects_time[0].text
        list_content=[]
        selects_content = soup.select("div#story_body_content > p")
        for c in selects_content:
            if len(c) != 0: #去除空值
                list_content.append(c.text)
        dic["content"] = list_content
        return dic
    except:
        return dic
list_newsss = []    
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    for data1 in executor.map(udn_everynews, udn_everynews_url):
        list_newsss.append(data1)
df = pd.DataFrame(list_newsss)
df.to_csv("udn_news.csv")

### 時間格式

In [ ]:
from datetime import datetime, timedelta, date
_format = "%Y/%m/%d"
d = datetime.strptime("2016/11/25", _format) ##2016/11/25 00:00:00
delta = timedelta(days = 1)
while d <= datetime.strptime(date.today().strftime(_format), _format): ##用striftime轉換date.today()成字串，再用轉成strptime比較大小
    print(d.strftime(_format)) ##把後面的小時、分鐘、秒去掉
    d += delta

In [12]:
from datetime import datetime, timedelta, date
_format = "%Y/%m/%d"
a = date.today()
b = datetime.strptime(date.today().strftime(_format), _format)
c = datetime.strptime("2016/11/24", _format)
print(a)
print(b)
print(c)
# print(a > b) ###所以說要用datetime.strptime轉換成時間格式才能比大小
print(b > c)

2017-11-25
2017-11-25 00:00:00
2016-11-24 00:00:00
True


In [141]:
date = datetime.datetime.strptime("2016/11/24", "%Y/%m/%d")

In [143]:
date.strftime("%Y/%m/%d")

'2016/11/24'